In [2]:

# Desafio de POO
# El objetivo es crear una clase para la conexión a la base de datos.

# Un método para crear la conexión para Mysql y otro para Postgresql, devuelve la conexión
# Piense en la contraseña y la seguridad del usuario
# Un método separado para devolver el cursor a cualquier base de datos.
# Un método para probar la conexión a cualquier base de datos, devuelve éxito o fracaso.
# Un método para seleccionar para cualquier base de datos, devuelve el resultado de la selección.
# Un método que devuelve el resultado seleccionado en el dataframe de pandas
# Un método para insertar en cualquier base de datos y devuelve éxito o no.

In [53]:
import os
import psycopg2
import pymysql
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [55]:
class ConnectionDb:
    def __init__(self, type_connection, host, port, user, password, database):       
        """Inicializa la conexión a la base de datos."""
        
        self.type_connection = type_connection
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.connection = None


        if self.type_connection == 'postgres':
            try:
                self.connection = psycopg2.connect(
                    host=self.host,
                    port=self.port,
                    user=self.user,
                    password=self.password,
                    database=self.database
                )
                print('Conexión Postgres establecida exitosamente')
            except psycopg2.DatabaseError as e:
                print(f'Ha ocurrido un error al intentar conectar a la Base de Datos Postgres, error: {e}')
            
        elif self.type_connection == 'mysql':
            try:
                self.connection = pymysql.connect(
                    host=self.host,
                    port=self.port,
                    user=self.user,
                    password=self.password,
                    database=self.database
                )
                print('Conexión MySQL establecida exitosamente')
            except pymysql.Error as e:
                print(f'Ha ocurrido un error al intentar conectar a la Base de Datos MySQL, error: {e}')
        else:
            print('Tipo de conexión no válido')
      

            
    
    def create_cursor(self, connection):
        cursor = connection.cursor()
        return cursor

    
    def test_connection(self, connection):
        """Verifica si la conexión a la base de datos está establecida."""
        
        if connection is not None:
            print('La conexión fue establecida con éxito')
        else:
            print('La conexión no fue establecida')
            

    def get_data_to_db(self, connection, table, columns, condition=''):
        """Obtiene los datos de la base de datos."""
        try:
            cursor = self.create_cursor(connection)
            sql = f'SELECT {columns} FROM {table}'
            if condition:
                sql += f' WHERE {condition};'
                
            cursor.execute(sql)
            results = cursor.fetchall()
            return results
        finally:
            cursor.close()

    
    def result_to_dataframe(self, result):
      """Convierte los resultados de la consulta en un DataFrame de Pandas."""
      df = pd.DataFrame(result) 
      return df
        

    def insert_data_to_db(self, connection, table, columns, values):
        """Inserta datos en la base de datos."""
        try:
            cursor = self.create_cursor(connection)            
            sql = f'INSERT INTO {table} {columns} VALUES {values}'
            cursor.execute(sql)
            connection.commit()
            print('Datos insertados exitosamente')
        except Exception as e:
            print(f'Error al insertar datos: {e}')
            connection.rollback()
        finally:
            cursor.close()

# Prueba Postgres

## DDL:

### CREATE TABLE poo_conexiones.mascotas (
    id serial4 NOT NULL,
    nombre varchar(50) NULL,
    fecha_nacimiento date NULL,
    especie varchar(100) NULL,
    color varchar(40) NULL,
    email varchar(255) NULL,
    CONSTRAINT prueba_pkey PRIMARY K Y 



id)
);
 KEY (id)
);

In [56]:
# Cargar variables de entorno para la conexión Postgres
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
user = os.getenv('POSTGRES_USER')
pswd = os.getenv('POSTGRES_PASSWORD')
db = os.getenv('POSTGRES_DB')

# Crear instancia de la clase ConnectionDb para la conexión Postgres
postgres = ConnectionDb('postgres', host, port, user, pswd, db)

# Probar la conexión
postgres.test_connection(postgres.connection)

# Obtener datos de la tabla 'poo_conexiones.mascotas'
results = postgres.get_data_to_db(postgres.connection, 'poo_conexiones.mascotas', '*')
# print('Datos obtenidos:', results)

# Convertir resultados a DataFrame de Pandas
df_pandas = postgres.result_to_dataframe(results)
# print('DataFrame de Pandas: ', df_pandas)

# Definir las columnas y valores para insertar en la tabla
columns = f'(nombre, fecha_nacimiento, especie, color)'
values = f"('Ravena', '21-02-2022', 'canina', 'marron, como el dulce de leche'),('Cocoa', '01-03-2014', 'canino', 'negro y canoso como una noche estrellada'), ('Sopa', '01-01-2020', 'equina', 'café con leche')"

# Insertar datos en la tabla 'poo_conexiones.mascotas'
postgres.insert_data_to_db(postgres.connection, 'poo_conexiones.mascotas', columns, values)

Conexión Postgres establecida exitosamente
La conexión fue establecida con éxito
Datos insertados exitosamente


# Prueba MySQL

## DDL

### CREATE TABLE poo_conexiones.mascotas (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(50) NULL,
    fecha_nacimiento DATE NULL,
    especie VARCHAR(100) NULL,
    color VARCHAR(40) NULL,
    email VARCHAR(255) NULL
);


In [58]:
# Cargar variables de entorno para la conexión MySQL
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
pswd = os.getenv('MYSQL_PASSWORD')
db = os.getenv('MYSQL_DB')

# Crear instancia de la clase ConnectionDb para la conexión MySQL
mysql = ConnectionDb('mysql', host, int(port), user, pswd, db)

# Probar la conexión
mysql.test_connection(mysql.connection)

# Obtener datos de la tabla 'poo_conexiones.mascotas'
results = mysql.get_data_to_db(mysql.connection, 'poo_conexiones.mascotas', '*', 'nombre="Ravena"' )
# print('Datos obtenidos:', results)

# Convertir resultados a DataFrame de Pandas
df_pandas = mysql.result_to_dataframe(results)
# print('DataFrame de Pandas: ', df_pandas)

# Definir las columnas y valores para insertar en la tabla
columns = f'(nombre, fecha_nacimiento, especie, color)'
values = f"('Ravena', '2022-02-21', 'canina', 'marron, como el dulce de leche'),('Cocoa', '2014-03-01', 'canino', 'negro y canoso como una noche estrellada'), ('Sopa', '2020-01-01', 'equina', 'café con leche')"


# Insertar datos en la tabla 'poo_conexiones.mascotas'
mysql.insert_data_to_db(mysql.connection, 'poo_conexiones.mascotas', columns, values)




Conexión MySQL establecida exitosamente
La conexión fue establecida con éxito
Datos insertados exitosamente


## Carga de variables de entorno en un archivo .env### Inserte sus credenciales a continuación

In [36]:
%%writefile .env
MYSQL_DB=*****
MYSQL_HOST=*****
MYSQL_PASSWORD=****
MYSQL_PORT=****
MYSQL_USER=****
POSTGRES_DB=****
POSTGRES_HOST=****
POSTGRES_PASSWORD=****
POSTGRES_PORT=***
POSTGRES_USER=****

Overwriting .env
